# Multi-model UNSEEN spatial analysis figures

This notebook can be run using papermill, for example:

```bash
papermill -p metric txx -p obs_config_file AGCD-CSIRO_r05_tasmax_config.mk -p obs AGCD spatial_analysis_multimodel.ipynb project-txx/spatial_analysis_multimodel_txx.ipynb

papermill -p metric rx1day -p obs_config_file AGCD-CSIRO_r05_precip_config.mk -p obs AGCD -p bc multiplicative spatial_analysis_multimodel.ipynb project-rx1day/spatial_analysis_multimodel_rx1day.ipynb
```

In [ ]:
import matplotlib.pyplot as plt  # noqa
import numpy as np
from pathlib import Path
import xarray as xr

from spatial_plots_multimodel import *

In [ ]:
# Default parameters
bc = None

In [ ]:
# Required parameters
kwargs = locals()
assert "metric" in kwargs, "Must provide a metric name (e.g., txx)"
assert (
    "obs_config_file" in kwargs
), "Must provide a obs_config_file name (e.g., AGCD-tasmax_config.mk)"
assert "obs" in kwargs, "Must provide a observation data set name (e.g., AGCD)"

In [ ]:
if metric == "txx":
    models = np.array(
        [
            "CAFE",
            "BCC-CSM2-MR",
            "CanESM5",
            "CMCC-CM2-SR5",
            "EC-Earth3",
            "IPSL-CM6A-LR",
            "MIROC6",
            "MPI-ESM1-2-HR",
            "MRI-ESM2-0",
            "NorCPM1",
        ]
    )
elif metric == "rx1day":
    models = np.array(
        [
            "BCC-CSM2-MR",
            "CanESM5",
            "CMCC-CM2-SR5",
            "EC-Earth3",
            "HadGEM3-GC31-MM",
            "IPSL-CM6A-LR",
            "MIROC6",
            "MPI-ESM1-2-HR",
            "MRI-ESM2-0",
            "NorCPM1",
        ]
    )

In [ ]:
# Get variables from makefile (nested dictionary)
var_dict = get_makefile_vars(models, metric, obs, obs_config_file=obs_config_file)

In [ ]:
# Filestem for figures and datatree
filestem = f"{metric}_{var_dict[obs]['timescale']}_{var_dict[obs]['region']}"
if bc is not None:
    filestem += f"_bias-corrected-{var_dict[obs]['obs_dataset']}-{bc}"
dt_file = f"{var_dict[obs]['project_dir']}/data/datatree_{filestem}.nc"

In [ ]:
# Extract some variables from the dictionaries
plot_dict = eval(var_dict[obs]["plot_dict"])
var = var_dict[obs]["var"]
time_agg = var_dict[obs]["time_agg"]
covariate_base = int(var_dict[obs]["covariate_base"])
covariates = eval(var_dict[obs]["gev_trend_period"])
plot_dict["fig_dir"] = Path(var_dict[obs]["fig_dir"]) / "multimodel"
plot_dict["filestem"] = filestem
plot_dict["filestem_no_bc"] = filestem.split("_bias")[0]
plot_dict["models"] = models

plot_dict_avg = plot_dict.copy()
stability_kwargs = {}
stability_anom_kwargs = {}

if metric == "txx":
    std_dev_ticks = np.arange(0, 4.5, 0.5)  # Median absolute deviation
    if bc is None:
        plot_dict_avg["ticks"] = np.arange(22, 52 + 4, 4)
        plot_dict["ticks"] = np.arange(24, 66 + 4, 4)
    else:
        plot_dict_avg["ticks"] = np.arange(22, 52 + 4, 4)
        plot_dict["ticks"] = np.arange(32, 56 + 2, 2)

    stability_anom_kwargs = dict(
        ticks=np.arange(-3.3, 3.5, 0.2),
        ticklabels=np.around(np.arange(-3.2, 3.4, 0.2), 1),
    )

elif metric == "rx1day":
    std_dev_ticks = np.arange(0, 50 + 10, 10)  # Median absolute deviation
    if bc is None:
        plot_dict_avg["ticks"] = np.arange(0, 200 + 20, 20)
        plot_dict["ticks"] = np.arange(0, 400 + 50, 50)
    else:
        plot_dict_avg["ticks"] = np.arange(0, 120 + 20, 20)
        plot_dict_avg["ticks_anom"] = np.arange(-11, 11 + 2, 2)
        plot_dict["ticks"] = np.arange(0, 450 + 50, 50)
        plot_dict["ticks_anom"] = np.arange(-170, 170 + 40, 40)

In [ ]:
# Create/open datatree of all model and observation datasets
if Path(dt_file).exists():
    dt = xr.open_datatree(dt_file)
else:
    # Create a data tree using dict of {model: filenames["metric_fcst"]}
    data_dict = {}
    data_dict[f"obs/{obs}"] = open_obs_dataset(var_dict, obs)
    for i, model in enumerate(models):
        print(f"{i}. {model}")
        data_dict[f"model/{model}"] = open_model_dataset(var_dict[model], bc)
    dt = xr.DataTree.from_dict(data_dict)
    dt.to_netcdf(dt_file, compute=True)

dt

In [ ]:
# Create nested dict of class instance containing variables and datasets
info = {}
info[obs] = InfoSet(
    name=obs,
    file=var_dict[obs]["metric_obs"],
    obs_name=obs,
    ds=dt[f"obs/{obs}"].ds,
    obs_ds=dt[f"obs/{obs}"].ds,
    bias_correction=bc,
    pval_mask=None,
    **plot_dict,
)

for m in models:
    info[m] = InfoSet(
        name=m,
        obs_name=obs,
        file=var_dict[m]["metric_fcst"],
        ds=dt[f"model/{m}"].ds,
        obs_ds=subset_obs_dataset(dt[f"obs/{obs}"].ds, dt[f"model/{m}"].ds),
        pval_mask=dt[f"model/{m}"].ds.pval_mask,
        bias_correction=bc,
        **plot_dict,
    )
    info[m].regridder = shared_grid_regridder(info[m].ds, method="conservative")

for m in info.keys():
    info[m].gev_mask = get_gev_mask(info, m, var_dict, test=var_dict[m]["gev_test"])

## Plots


### Independence and stability

In [ ]:
if bc is None:
    # Independent lead time
    plot_min_independent_lead(info, plot_dict)

In [ ]:
if bc is None:
    # Stability (don't plot for diff bc)
    for method in ["aep", "median"]:
        plot_stability(info, var_dict, plot_dict, method, anomaly=False)
        plot_stability(
            info,
            var_dict,
            plot_dict,
            method=method,
            anomaly=True,
            **stability_anom_kwargs,
        )

### Metric maximum and median

In [ ]:
plot_time_agg(info, var, "maximum", plot_dict)

In [ ]:
plot_time_agg(info, var, "median", plot_dict_avg)

### Metric maximum (subsampled)

In [ ]:
plot_time_agg_subsampled(info, obs, "maximum", plot_dict, 10000)

### Model minus observation anomalies

In [ ]:
for anom in ["anom", "anom_pct", "anom_std", "anom_2000yr"]:
    plot_obs_anom(info, obs, var, "maximum", anom, covariate_base, plot_dict)

In [ ]:
plot_obs_anom(
    info,
    obs,
    var,
    "median",
    "anom",
    covariate_base,
    plot_dict_avg,
)

### Seasonality/event year

In [ ]:
cmap = month_cmap_alt if metric == "txx" else month_cmap

In [ ]:
plot_event_month_mode(
    info, plot_dict, cmap=cmap, add_labels=True if metric == "txx" else False
)

In [ ]:
plot_record_event_month(info, plot_dict, time_agg=time_agg, cmap=cmap)

In [ ]:
plot_event_year(info, var, time_agg, plot_dict, ticks=np.arange(1960, 2025, 5))

In [ ]:
plot_metric_variability(info, var, plot_dict, ticks=std_dev_ticks)

### GEV parameters

In [ ]:
# GEV/empirical
for param in ["c", "location_0", "location_1", "scale_0", "scale_1"]:
    plot_nonstationary_gev_param(info, param, plot_dict)

### Annual exceedance probability (AEP)

In [ ]:
aep = 1

In [ ]:
plot_aep_empirical(info, plot_dict, var, aep=aep)

In [ ]:
plot_aep(info, plot_dict, covariates[-1], aep=aep)

In [ ]:
plot_aep_trend(
    info,
    plot_dict,
    covariates,
    aep=aep,
)

# Cumulative annual exceedance probability

In [ ]:
plot_abstract(
    info, plot_dict, time_agg=time_agg, start_year=2025, n_years=10
)  # For graphic abstract (multi-model median only)

In [ ]:
plot_new_record_probability_empirical(
    info, plot_dict, var, time_agg=time_agg, n_years=10
)

In [ ]:
plot_new_record_probability(
    info, plot_dict, covariate_base, time_agg=time_agg, n_years=10
)

In [ ]:
plot_obs_ari(
    info,
    plot_dict,
    var,
    obs,
    covariate_base,
    time_agg=time_agg,
)